# Saving Materials Project data

In this notebook, we run through the elements that we need unary and binary oxide formation energies for.

In [1]:
import pickle

import numpy as np
import matplotlib.pyplot as plt

from collections import defaultdict

from pymatgen.ext.matproj import MPRester

# Initialize the MP Rester
mpr = MPRester('pn8XdbGhMrv90STu')


In [2]:
elements =  [ "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Nb", "Mo", 
             "Li", "Be", "Na", "Mg", "K", "Ca", "Rb", "Sr", "Cs", "Ba", # Alkalis
            "Sc", "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn",
            "Ga", "Ge", "As", "Se", "Br",
            "Y", "Zr", "Nb", "Mo", "Tc", "Ru", "Rh", "Pd", "Ag", "Cd",
            "In", "Sn", "Sb", "Te", "I",
            "Hf", "Ta", "W", "Re", "Os", "Ir", "Pt", "Au", "Hg", 
            "Tl", "Pb", "Bi",
           "La", "Ce", "Nd", "Pr", "Sm", "Eu", "Gd", "Tb", "Dy", "Ho", "Er", "Tm", "Yb", "Lu"]

ele2gs = defaultdict()

for ele in elements:
    entries = mpr.get_entries(ele)
    for entry in entries:
        ene_per_atom = entry.energy/list(dict(entry.composition).values())[0]
        if ele not in ele2gs.keys():
            ele2gs[ele] = ene_per_atom
        elif ene_per_atom<ele2gs[ele]:
            ele2gs[ele] = ene_per_atom
            
pickle.dump(ele2gs, open("ele2gs.p", "wb"))

ele2gs["O"] = -4.95

In [3]:
def calc_formation_ene(energy_per_atom, composition):
    """
    Need to calculate formation energies ourselves because materials project formation energies are inconsistent.
    See https://matsci.org/t/formation-energy-calculation/41574 for further information.
    
    """    
    total_atoms = sum(composition.values())
    formation_energy = energy_per_atom
    for element in composition:
        formation_energy-=ele2gs[element]*composition[element]/total_atoms
    return formation_energy

def get_chemsys_data(chemsys):
    query = mpr.get_data(chemsys)
    material_ids = [q['material_id'] for q in query]
    material_names = [q['pretty_formula'] for q in query]
    material_enes = [q['energy_per_atom'] for q in query]
    material_comps = [q['unit_cell_formula'] for q in query]
    
    material_names_ = []
    material_ids_ = []
    form_enes_ = []
    for idx, energy in enumerate(material_enes):
        ref_form_ene = calc_formation_ene(energy, material_comps[idx])
        form_enes_.append(ref_form_ene)
        material_names_.append(material_names[idx])
        material_ids_.append(material_ids[idx])

    return material_ids_, material_names_, form_enes_

# takes about 20 minutes to run this

In [4]:
binary_oxide_data = defaultdict()
unary_oxide_data = defaultdict()

for idx, ele in enumerate(elements):
    if ele not in unary_oxide_data.keys():
        # save unary for ele, saves iterating over the same oxides
        ids_, names_, form_enes = get_chemsys_data('{}-O'.format(ele))
        
        assert len(ids_)==len(names_)==len(form_enes)
        
        unary_oxide_data[ele] = {
            "names": names_,
            "mp_ids": ids_,
            "formation_energies": form_enes
        }
        pickle.dump(unary_oxide_data, open("unary_oxide_data.p", "wb"))
     
    for ele2 in elements[idx:]:
        # save unary for ele2
        if ele2 not in unary_oxide_data.keys():
            ids_, names_, form_enes = get_chemsys_data('{}-O'.format(ele2))

            assert len(ids_)==len(names_)==len(form_enes)

            unary_oxide_data[ele2] = {
                "names": names_,
                "mp_ids": ids_,
                "formation_energies": form_enes
            }
            pickle.dump(unary_oxide_data, open("unary_oxide_data.p", "wb"))
        
        
        if ele+"_"+ele2 in binary_oxide_data.keys() or ele==ele2:
            continue    
        # save binary
        ids_, names_, form_enes = get_chemsys_data('{}-{}-O'.format(ele, ele2))

        assert len(ids_)==len(names_)==len(form_enes)

        binary_oxide_data[ele+"_"+ele2] = {
            "names": names_,
            "mp_ids": ids_,
            "formation_energies": form_enes
        }
        pickle.dump(unary_oxide_data, open("binary_oxide_data.p", "wb"))

